# Embeddings

In [ ]:
from src.ncrna.tasks.lm.drnafm import EvoFlow
import torch

model = EvoFlow.load_from_pretrained('weights/mini-v1.ckpt').to('cuda:0')
tokenizer = model.alphabet

sequences = ['GGGGGUGUAGCUCAGUGGUAGAGCGCGUGCUUAGCAUGUACGAGGUCCCGGGUUCAAUCCCCGGCACCUCCA'] # tRNA-Ala
x = torch.tensor(tokenizer.batch_tokenize(sequences), dtype=torch.int64, device='cuda:0')

with torch.cuda.amp.autocast(dtype=torch.float16):
    logits, hidden_states = model(x, return_last_hidden_state=True)

print(hidden_states.shape)

# Generation

Unconditional

In [ ]:
# Unconditional generation is supported using the sample() method directly.

from src.ncrna.sampling.sampling import parse_prompt

xt = parse_prompt(None, 1, tokenizer, 'cuda:0', 100) # Unconditionally sampling 1 RNA of length 100

steps = 100
tau = 0.1

model.sample(xt, steps, tau)

Conditional

In [ ]:
# In our preprint, we construct scaffolds form the 1RAW aptamer binding recognition site. To attempt that same experiment using the mini model:

prompt = '......GGAAG.........................' # Conditional generation requires specifying a prompt, which uses the '.' character for the <mask> token
xt = parse_prompt(prompt, 1, tokenizer, 'cuda:0', 100)
model.sample(xt, steps, tau)